## WGU Capstone - Brennan Laird - Stock Price Forecasting

In [1]:
# import statements for Python code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import math
import pandas_ta
import itertools
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [16]:
# Additional import statements specific to the ML portion
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg

from statsmodels.graphics.tsaplots import plot_acf
#from pmdarima.arima import auto_arima



In [17]:
#imports for use in creating the web app portion
import ipywidgets as widgets
from ipywidgets import interactive,interact
from IPython.display import display, clear_output

In [18]:
def get_data(ticker):
	# Import the CSV file 
	#This should be converted to a function that will be called based on the ticker symbol passed via the Anvil code.

	path = 'C:\\Users\\brenn\\Documents\\School\\C964 - Capstone\\Datasets\\Stocks and ETF\\Stocks\\'

	extension = '.us.txt'
	file_name = path + ticker + extension

	# Read the specified price into a dataframe
	price_data = pd.read_csv(file_name, index_col = 'Date', parse_dates=['Date'])
	
	price_fig = go.Figure()

	price_fig.add_trace(go.Scatter(x=list(price_data.index), y=list(price_data['Close'])))

	title = ticker + " Price"
	price_fig.update_layout(title_text = title)


	# Add range slider
	price_fig.update_layout(
		xaxis=dict(
			rangeselector=dict(
				buttons=list([
					dict(count=1,
						label="1m",
						step="month",
						stepmode="backward"),
					dict(count=6,
						label="6m",
						step="month",
						stepmode="backward"),
					dict(count=1,
						label="YTD",
						step="year",
						stepmode="todate"),
					dict(count=1,
						label="1y",
						step="year",
						stepmode="backward"),
					dict(step="all")
				])
			),
			rangeslider=dict(
				visible=True
			),
			type="date"
		)
	)

	price_fig.show()
	
	
	
	vol_fig = go.Figure()

	vol_fig.add_trace(go.Scatter(x=list(price_data.index), y=list(price_data['Volume'])))

	title = ticker + " Volume"
	vol_fig.update_layout(title_text = title)
	
		# Add range slider
	vol_fig.update_layout(
		xaxis=dict(
			rangeselector=dict(
				buttons=list([
					dict(count=1,
						label="1m",
						step="month",
						stepmode="backward"),
					dict(count=6,
						label="6m",
						step="month",
						stepmode="backward"),
					dict(count=1,
						label="YTD",
						step="year",
						stepmode="todate"),
					dict(count=1,
						label="1y",
						step="year",
						stepmode="backward"),
					dict(step="all")
				])
			),
			rangeslider=dict(
				visible=True
			),
			type="date"
		)
	)

	vol_fig.show()
	
	# Adding the 50-day and 200-day moving averages to the dataframe
	price_data['50dayMA'] = price_data.Close.rolling(50).mean()
	price_data['200dayMA'] = price_data.Close.rolling(200).mean()
	
	ma_fig = go.Figure()

	ma_fig.add_trace(go.Scatter(x=list(price_data.index), y=list(price_data['Close'])))
	ma_fig.add_trace(go.Scatter(x=list(price_data.index), y=list(price_data['50dayMA'])))
	ma_fig.add_trace(go.Scatter(x=list(price_data.index), y=list(price_data['200dayMA'])))


	title = ticker + " Moving Averages"
	ma_fig.update_layout(title_text = title)


	# Add range slider
	ma_fig.update_layout(
		xaxis=dict(
			rangeselector=dict(
				buttons=list([
					dict(count=1,
						 label="1m",
						 step="month",
						 stepmode="backward"),
					dict(count=6,
						 label="6m",
						 step="month",
						 stepmode="backward"),
					dict(count=1,
						 label="YTD",
						 step="year",
						 stepmode="todate"),
					dict(count=1,
						 label="1y",
						 step="year",
						 stepmode="backward"),
					dict(step="all")
				])
			),
			rangeslider=dict(
				visible=True
			),
			type="date"
		)
	)

	ma_fig.show()
	
	
	price_data.ta.ema(close='Close', length = 10, append = True)
	
	price_data_trunc = price_data.iloc[10:]
	
	

In [19]:
ticker_box = widgets.Dropdown(options=['','AAPL', 'GME'])
ticker_box.value = ''
display(ticker_box)

Dropdown(options=('', 'AAPL', 'GME'), value='')

In [6]:
def on_data_click(b):
    with out:
        clear_output()
        ticker = ticker_box.value
        if ticker == '':
            #Do nothing
            print("Nothing selected")
            return
        else:
            get_data(ticker)
            return ticker

In [7]:
data_button = widgets.Button(description='Get Data', disabled=False)
out = widgets.Output()
#ticker = ticker_box.value

        
# linking button and function together using a button's method
data_button.on_click(on_data_click)

widgets.VBox([data_button,out])

## Adding the EMA to the dataset

## Rolling Forecast ARIMA Model with EMA Data

In [8]:
# #Split the data into training and testing sets

# train_set, test_set = price_data_trunc[0:int(len(price_data_trunc)*0.66)], price_data_trunc[int(len(price_data_trunc)*0.66):]

# train_array = train_set['EMA_10'].values
# test_array = test_set['EMA_10'].values
# predictions = []

In [9]:
# history = price_data_trunc[0:len(price_data_trunc)]

In [10]:
#history.info()

In [11]:
# # Puts the EMA prices into a list
# if ticker_box.value != '':
#     history = [i for i in price_data_trunc['EMA_10']]

#     #Puts the history into a dataframe
#     history_df = pd.DataFrame(history)

#     #Create a 30 day rolling forecat
#     for i in tqdm(range(30)):
#         # Runs and fits the ARIMA model with the specified order
#         model = ARIMA(history, order=(2,1,3))
#         model_fit = model.fit()
    
#         # Uses the forecast method to predict a single future timestep
#         next_forecast = model_fit.forecast()
    
#         # Gets the value of the next prediction
#         next_pred = next_forecast[0]
    
    
#         #pd.concat(history,next_forecast)
#         history.append(next_pred)
#         predictions.append(next_pred)
    
#         #print("Loop ", i, " prediction : ", next_pred)
# else:
#     print("No Ticker selected")

In [12]:
# pred_df = pd.DataFrame(predictions)
# pred_df.index = pred_df.index + len(history)-30

In [13]:
#pred_df.info()

In [14]:
# ax = history_df.plot( xlim=[len(history_df)-365, len(history_df)+len(pred_df)], figsize = (20,10))
# pred_df.plot(ax = ax)

## Below loop is used to find the minimum AIC Value

In [15]:
# history = [i for i in price_data_trunc['EMA_10']]
# #Setting up the ranges to search within
# p=q=range(10,16)
# d=range(0,2)

# #Creates a list of all combinations within the specified range
# pdq = list(itertools.product(p,d,q))

# # Set the minimum to a high number
# min_aic = 1000000

# #Iterate through each item in the list to find the minimum value of AIC
# for item in tqdm(pdq):
#     model = ARIMA(history, order=item)
#     model_fit = model.fit()
#     if model_fit.aic < min_aic:
#         min_aic = model_fit.aic
#         item_best = item
#         #print("Best so far: ", item_best, " AIC: ", min_aic )  
# print(item_best, " AIC: ", min_aic ) 